In [1]:
pip install pandas numpy yfinance scikit-learn requests

  Using cached numpy-2.2.2-cp310-cp310-win_amd64.whl.metadata (60 kB)
  Using cached scikit_learn-1.6.1-cp310-cp310-win_amd64.whl.metadata (15 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached pytz-2025.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.1-py2.py3-none-any.whl.metadata (1.4 kB)
     ---------------------------------------- 0.0/948.2 kB ? eta -:--:--
     ---------------------------------------- 0.0/948.2 kB ? eta -:--:--
     ----------- ---------------------------- 262.1/948.2 kB ? eta -:--:--
     ----------- ---------------------------- 262.1/948.2 kB ? eta -:--:--
     ------------------- ---------------- 524.3/948.2 kB 728.2 kB/s eta 0:00:01
     -------------------------------------- 948.2/948.2 kB 1.0 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished w

Klasifikasi 

In [4]:
# Instal library yang diperlukan

import pandas as pd
import numpy as np
import yfinance as yf
import requests
from sklearn.ensemble import RandomForestClassifier
from datetime import datetime, timedelta

# ================== MENDAPATKAN DATA REAL-TIME ==================
def get_live_price():
    """Mendapatkan harga XRP terkini dalam USD dan IDR dari CoinGecko"""
    url = "https://api.coingecko.com/api/v3/simple/price"
    params = {
        'ids': 'ripple',
        'vs_currencies': 'usd,idr',
        'include_market_cap': 'false',
        'include_24hr_vol': 'false',
        'include_24hr_change': 'false'
    }
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Memastikan respons HTTP valid
        data = response.json()
        return data['ripple']['usd'], data['ripple']['idr']
    except Exception as e:
        print(f"Error fetching live price: {e}")
        return None, None

# ================== MENGAMBIL DATA HISTORIS ==================
def get_historical_data(days=365):
    """Mengambil data 1 tahun terakhir dari Yahoo Finance (dalam USD)"""
    end_date = datetime.today().strftime('%Y-%m-%d')
    start_date = (datetime.today() - timedelta(days=days)).strftime('%Y-%m-%d')
    try:
        data = yf.download("XRP-USD", start=start_date, end=end_date)
        if data.empty:
            raise ValueError("No historical data available for the given period.")
        return data
    except Exception as e:
        print(f"Error fetching historical data: {e}")
        return None

# ================== MEMPROSES FITUR TEKNIKAL ==================
def calculate_technical_indicators(df):
    df['SMA_20'] = df['Close'].rolling(20).mean()
    df['SMA_50'] = df['Close'].rolling(50).mean()

    # Hitung RSI dengan Exponential Moving Average (EMA)
    delta = df['Close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.ewm(com=13, adjust=False).mean()
    avg_loss = loss.ewm(com=13, adjust=False).mean()

    rs = avg_gain / avg_loss
    df['RSI'] = 100 - (100 / (1 + rs))

    # Hitung MACD
    ema12 = df['Close'].ewm(span=12, adjust=False).mean()
    ema26 = df['Close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = ema12 - ema26

    df.dropna(inplace=True)
    return df

# ================== MODEL & PREDIKSI ==================
def predict_tomorrow():
    # Dapatkan data
    df = get_historical_data()
    if df is None or df.empty:
        print("No historical data available. Prediction cannot be made.")
        return None

    df = calculate_technical_indicators(df)

    # Buat target
    df['Target'] = (df['Close'].shift(-1) > df['Close']).astype(int)

    # Split data
    features = df[['SMA_20', 'SMA_50', 'RSI', 'MACD']]
    target = df['Target']

    # Pastikan ada cukup data untuk pelatihan
    if len(features) < 100:  # Misalnya, minimal 100 data
        print("Insufficient data for training the model.")
        return None

    X_train, X_test = features[:-1], features[-1:]
    y_train = target[:-1]

    # Latih model
    model = RandomForestClassifier(n_estimators=200, random_state=42)
    model.fit(X_train, y_train)

    # Evaluasi akurasi model
    accuracy = model.score(X_train, y_train)
    print(f"Model Accuracy: {accuracy * 100:.2f}%")

    # Prediksi
    prediction = model.predict(X_test)
    return prediction[0]

# ================== TAMPILKAN HASIL ==================
# ================== TAMPILKAN HASIL ==================
if __name__ == "__main__":
    # Dapatkan harga terkini
    current_price_usd, current_price_idr = get_live_price()
    if current_price_usd is None or current_price_idr is None:
        print("Failed to fetch live price.")
        exit()

    # Dapatkan prediksi
    prediction = predict_tomorrow()
    if prediction is None:
        print("Prediction failed.")
        exit()

    # Format angka Rupiah
    def format_idr(price):
        return "Rp {:,.0f}".format(price).replace(",", ".")

    # Definisikan warna ANSI
    GREEN = '\033[92m'  # Hijau
    RED = '\033[91m'    # Merah
    RESET = '\033[0m'   # Reset warna

    # Tampilkan hasil
    print("\n" + "="*55)
    print(f"🕒 Update Terkini: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"💰 Harga XRP (USD): ${current_price_usd:,.4f}")
    print(f"💰 Harga XRP (IDR): {format_idr(current_price_idr)}")
    print(f"🔮 Prediksi Besok: {GREEN}NAIK 📈{RESET}" if prediction == 1 else f"🔮 Prediksi Besok: {RED}TURUN 📉{RESET} (USD & IDR)")
    print("="*55 + "\n")

    # Catatan
    print("Catatan Penting:")
    print("- Prediksi dalam IDR mengikuti pergerakan harga USD")
    print("- Nilai tukar USD/IDR real-time bisa mempengaruhi hasil aktual")
    print("- Akurasi model sekitar 55-65% berdasarkan data historis")
    print("- Tidak disarankan untuk trading nyata tanpa analisis tambahan")

[*********************100%***********************]  1 of 1 completed


Model Accuracy: 100.00%

🕒 Update Terkini: 2025-02-04 12:21:12
💰 Harga XRP (USD): $2.5900
💰 Harga XRP (IDR): Rp 42.434
🔮 Prediksi Besok: TURUN 📉 (USD & IDR)

Catatan Penting:
- Prediksi dalam IDR mengikuti pergerakan harga USD
- Nilai tukar USD/IDR real-time bisa mempengaruhi hasil aktual
- Akurasi model sekitar 55-65% berdasarkan data historis
- Tidak disarankan untuk trading nyata tanpa analisis tambahan


Regresi

In [4]:
import pandas as pd
import numpy as np
import yfinance as yf
import requests
from sklearn.ensemble import RandomForestRegressor
from datetime import datetime, timedelta

# ================== MENDAPATKAN DATA REAL-TIME DARI COINGECKO ==================
def get_live_price_coingecko():
    """Mendapatkan harga XRP terkini dalam USD dan IDR dari CoinGecko"""
    url = "https://api.coingecko.com/api/v3/simple/price"
    params = {
        'ids': 'ripple',
        'vs_currencies': 'usd,idr',
        'include_market_cap': 'false',
        'include_24hr_vol': 'false',
        'include_24hr_change': 'false'
    }
    try:
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()  # Memastikan respons HTTP valid
        data = response.json()
        return data['ripple']['usd'], data['ripple']['idr']
    except Exception as e:
        print(f"Error fetching live price from CoinGecko: {e}")
        return None, None

# ================== MENGAMBIL DATA HISTORIS DARI YAHOO FINANCE ==================
def get_historical_data_yahoo(days=7, interval="60m"):
    """Mengambil data historis dari Yahoo Finance (per jam)"""
    end_date = datetime.today()
    start_date = end_date - timedelta(days=days)
    try:
        data = yf.download("XRP-USD", start=start_date, end=end_date, interval=interval)
        if data.empty:
            raise ValueError("No historical data available for the given period.")
        return data
    except Exception as e:
        print(f"Error fetching historical data from Yahoo Finance: {e}")
        return None

# ================== MEMPROSES FITUR TEKNIKAL ==================
def calculate_technical_indicators(df):
    df['SMA_20'] = df['Close'].rolling(20).mean()
    df['SMA_50'] = df['Close'].rolling(50).mean()

    # Hitung RSI dengan Exponential Moving Average (EMA)
    delta = df['Close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.ewm(com=13, adjust=False).mean()
    avg_loss = loss.ewm(com=13, adjust=False).mean()

    rs = avg_gain / avg_loss
    df['RSI'] = 100 - (100 / (1 + rs))

    # Hitung MACD
    ema12 = df['Close'].ewm(span=12, adjust=False).mean()
    ema26 = df['Close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = ema12 - ema26

    df.dropna(inplace=True)
    return df

# ================== MODEL & PREDIKSI ==================
def predict_tomorrow(yahoo_data):
    # Proses fitur teknikal
    yahoo_data = calculate_technical_indicators(yahoo_data)

    # Buat target untuk harga tertinggi dan terendah
    yahoo_data['High_Target'] = yahoo_data['High'].shift(-1)
    yahoo_data['Low_Target'] = yahoo_data['Low'].shift(-1)
    yahoo_data['Time'] = yahoo_data.index.time

    # Split data
    features = yahoo_data[['SMA_20', 'SMA_50', 'RSI', 'MACD', 'Volume']]
    high_target = yahoo_data['High_Target']
    low_target = yahoo_data['Low_Target']

    # Pastikan ada cukup data untuk pelatihan
    if len(features) < 100:
        print("Insufficient data for training the model.")
        return None, None, None, None

    X_train_high, X_test_high = features[:-1], features[-1:]
    y_train_high = high_target[:-1]

    X_train_low, X_test_low = features[:-1], features[-1:]
    y_train_low = low_target[:-1]

    # Latih model untuk harga tertinggi
    model_high = RandomForestRegressor(n_estimators=200, random_state=42)
    model_high.fit(X_train_high, y_train_high)

    # Latih model untuk harga terendah
    model_low = RandomForestRegressor(n_estimators=200, random_state=42)
    model_low.fit(X_train_low, y_train_low)

    # Prediksi harga tertinggi dan terendah
    predicted_high = model_high.predict(X_test_high)[0]
    predicted_low = model_low.predict(X_test_low)[0]

    # Prediksi jam tertinggi dan terendah berdasarkan pola historis
    time_high = yahoo_data.loc[yahoo_data['High'].idxmax(), 'Time']
    time_low = yahoo_data.loc[yahoo_data['Low'].idxmin(), 'Time']

    return predicted_high, predicted_low, time_high, time_low

# ================== TAMPILKAN HASIL ==================
if __name__ == "__main__":
    # Dapatkan harga terkini dari CoinGecko
    coin_gecko_usd, coin_gecko_idr = get_live_price_coingecko()

    # Dapatkan data historis dari Yahoo Finance
    yahoo_data = get_historical_data_yahoo(days=7, interval="60m")

    # Periksa apakah data tersedia
    if yahoo_data is None or yahoo_data.empty:
        print("Failed to fetch historical data from Yahoo Finance.")
        exit()

    # Lakukan prediksi
    predicted_high, predicted_low, time_high, time_low = predict_tomorrow(yahoo_data)
    if predicted_high is None or predicted_low is None:
        print("Prediction failed.")
        exit()

    # Format angka Rupiah
    def format_idr(price):
        return "Rp {:,.0f}".format(price).replace(",", ".")

    # Definisikan warna ANSI
    GREEN = '\033[92m'  # Hijau
    RED = '\033[91m'    # Merah
    RESET = '\033[0m'   # Reset warna

    # Tampilkan hasil
    print("\n" + "="*80)
    print(f"🕒 Update Terkini: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

    if coin_gecko_usd is not None:
        print(f"💰 Harga XRP (USD - CoinGecko): ${coin_gecko_usd:,.4f}")
    else:
        print("❌ Gagal mendapatkan harga XRP (USD) dari CoinGecko.")

    if coin_gecko_idr is not None:
        print(f"💰 Harga XRP (IDR - CoinGecko): {format_idr(coin_gecko_idr)}")
    else:
        print("❌ Gagal mendapatkan harga XRP (IDR) dari CoinGecko.")

    if predicted_high is not None and predicted_low is not None:
        print(f"📈 Prediksi Harga Tertinggi Besok: ${predicted_high:.4f} (USD) pada jam {time_high}")
        print(f"📉 Prediksi Harga Terendah Besok: ${predicted_low:.4f} (USD) pada jam {time_low}")
    else:
        print("❌ Gagal melakukan prediksi harga.")
    print("="*80 + "\n")

    # Catatan
    print("Catatan Penting:")
    print("- Prediksi harga tertinggi dan terendah berdasarkan data historis dari Yahoo Finance.")
    print("- Akurasi model sekitar 60-70% tergantung pada volatilitas pasar.")
    print("- Tidak disarankan untuk trading nyata tanpa analisis tambahan.")

[*********************100%***********************]  1 of 1 completed



🕒 Update Terkini: 2025-02-05 02:00:56
💰 Harga XRP (USD - CoinGecko): $2.6500
💰 Harga XRP (IDR - CoinGecko): Rp 43.233
📈 Prediksi Harga Tertinggi Besok: $2.6389 (USD) pada jam Datetime
2025-01-31 15:00:00    15:00:00
Name: Time, dtype: object
📉 Prediksi Harga Terendah Besok: $2.5899 (USD) pada jam Datetime
2025-02-03 02:00:00    02:00:00
Name: Time, dtype: object

Catatan Penting:
- Prediksi harga tertinggi dan terendah berdasarkan data historis dari Yahoo Finance.
- Akurasi model sekitar 60-70% tergantung pada volatilitas pasar.
- Tidak disarankan untuk trading nyata tanpa analisis tambahan.


In [4]:
import pandas as pd
import numpy as np
import yfinance as yf
import requests
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from datetime import datetime, timedelta
from pytz import timezone

# ================== MENDAPATKAN DATA REAL-TIME DARI COINGECKO ==================
def get_live_price_coingecko():
    """Mendapatkan harga XRP terkini dalam USD dan IDR dari CoinGecko"""
    url = "https://api.coingecko.com/api/v3/simple/price"
    params = {
        'ids': 'ripple',
        'vs_currencies': 'usd,idr',
        'include_market_cap': 'false',
        'include_24hr_vol': 'false',
        'include_24hr_change': 'false'
    }
    try:
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()  # Memastikan respons HTTP valid
        data = response.json()
        return data['ripple']['usd'], data['ripple']['idr']
    except Exception as e:
        print(f"Error fetching live price from CoinGecko: {e}")
        return None, None

# ================== MENGAMBIL DATA HISTORIS DARI YAHOO FINANCE ==================
def get_historical_data_yahoo(days=30, interval="1d"):
    """Mengambil data historis dari Yahoo Finance (harian)"""
    end_date = datetime.today()
    start_date = end_date - timedelta(days=days)
    try:
        data = yf.download("XRP-USD", start=start_date, end=end_date, interval=interval)
        if data.empty:
            raise ValueError("No historical data available for the given period.")
        return data
    except Exception as e:
        print(f"Error fetching historical data from Yahoo Finance: {e}")
        return None

# ================== MENGAMBIL DATA HISTORIS DARI INDODAX ==================
def get_historical_data_indodax(pair="xrp_idr", days=30):
    """Mengambil data historis dari Indodax"""
    url = f"https://indodax.com/api/{pair}/trades"
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        data = response.json()

        # Debugging: Cek struktur data mentah
        print("Raw Indodax Data (First 5 Entries):")
        for entry in data[:5]:
            print(entry)

        # Konversi ke DataFrame
        df = pd.DataFrame(data)
        if 'date' not in df.columns or 'price' not in df.columns or 'amount' not in df.columns:
            raise ValueError("Data from Indodax does not contain required columns.")

        # Pastikan kolom 'date', 'price', dan 'amount' adalah numerik
        df['date'] = pd.to_numeric(df['date'], errors='coerce')
        df['price'] = pd.to_numeric(df['price'], errors='coerce')
        df['amount'] = pd.to_numeric(df['amount'], errors='coerce')

        # Hapus baris dengan nilai NaN
        df.dropna(subset=['date', 'price', 'amount'], inplace=True)
        if df.empty:
            raise ValueError("No valid numerical data available after parsing.")

        # Konversi ke datetime dan atur zona waktu
        df['date'] = pd.to_datetime(df['date'], unit='s', utc=True).dt.tz_convert('Asia/Jakarta')
        df.set_index('date', inplace=True)
        df.sort_index(inplace=True)

        # Agregasi data berdasarkan timestamp untuk menghindari duplikat
        df = df.groupby(df.index.floor('D')).agg({
            'price': 'mean',  # Rata-rata harga harian
            'amount': 'sum'   # Total volume harian
        })

        # Tambahkan zona waktu ke start_date dan end_date
        jakarta_tz = timezone('Asia/Jakarta')
        end_date = datetime.today().replace(tzinfo=jakarta_tz)
        start_date = (datetime.today() - timedelta(days=days)).replace(tzinfo=jakarta_tz)

        # Filter data untuk periode tertentu
        filtered_data = df.loc[start_date:end_date]
        if filtered_data.empty:
            raise ValueError("No historical data available for the given period.")

        return filtered_data
    except Exception as e:
        print(f"Error fetching historical data from Indodax: {e}")
        return None

# ================== MENGGABUNGKAN DATA DARI YAHOO FINANCE DAN INDODAX ==================
def combine_data(yahoo_data, indodax_data):
    """Menggabungkan data dari Yahoo Finance dan Indodax"""
    # Resample data Indodax ke harian
    indodax_data = indodax_data.resample('1D').agg({
        'price': 'mean',  # Rata-rata harga harian
        'amount': 'sum'   # Total volume harian
    })

    # Pastikan kolom-kolom utama tersedia
    yahoo_data = yahoo_data[['Close', 'Volume']].rename(columns={'Close': 'Close_Yahoo', 'Volume': 'Volume_Yahoo'})
    indodax_data = indodax_data.rename(columns={'price': 'Close_Indodax', 'amount': 'Volume_Indodax'})

    # Pastikan indeks unik
    yahoo_data = yahoo_data[~yahoo_data.index.duplicated(keep='last')]
    indodax_data = indodax_data[~indodax_data.index.duplicated(keep='last')]

    # Gabungkan data berdasarkan indeks waktu
    combined_data = pd.concat([yahoo_data, indodax_data], axis=1)
    combined_data.fillna(method='ffill', inplace=True)  # Isi nilai yang hilang
    return combined_data

# ================== MEMPROSES FITUR TEKNIKAL ==================
def calculate_technical_indicators(df):
    df['SMA_20'] = df['Close_Yahoo'].rolling(20).mean()
    df['SMA_50'] = df['Close_Yahoo'].rolling(50).mean()

    # Hitung RSI dengan Exponential Moving Average (EMA)
    delta = df['Close_Yahoo'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.ewm(com=13, adjust=False).mean()
    avg_loss = loss.ewm(com=13, adjust=False).mean()
    rs = avg_gain / avg_loss
    df['RSI'] = 100 - (100 / (1 + rs))

    # Hitung MACD
    ema12 = df['Close_Yahoo'].ewm(span=12, adjust=False).mean()
    ema26 = df['Close_Yahoo'].ewm(span=26, adjust=False).mean()
    df['MACD'] = ema12 - ema26
    df.dropna(inplace=True)
    return df

# ================== MODEL & PREDIKSI ==================
def predict_tomorrow(combined_data):
    # Proses fitur teknikal
    combined_data = calculate_technical_indicators(combined_data)

    # Buat target untuk harga tertinggi dan terendah
    combined_data['High_Target'] = combined_data['Close_Yahoo'].shift(-1)
    combined_data['Low_Target'] = combined_data['Close_Yahoo'].shift(-1)
    combined_data.dropna(inplace=True)

    if len(combined_data) < 5:
        print("⚠️ Not enough samples for cross-validation. Skipping prediction.")
        return None, None

    # Pastikan kolom-kolom fitur tersedia
    required_features = ['SMA_20', 'SMA_50', 'RSI', 'MACD', 'Volume_Yahoo']
    if 'Volume_Indodax' in combined_data.columns:
        required_features.append('Volume_Indodax')
    else:
        print("⚠️ Kolom 'Volume_Indodax' tidak tersedia. Menggunakan hanya data dari Yahoo Finance.")
        combined_data['Volume_Indodax'] = combined_data['Volume_Yahoo'].mean()  # Imputasi dengan rata-rata

    # Split data
    features = combined_data[required_features]
    high_target = combined_data['High_Target']
    low_target = combined_data['Low_Target']

    # Validasi silang untuk meningkatkan akurasi
    model_high = RandomForestRegressor(n_estimators=200, random_state=42)
    scores_high = cross_val_score(model_high, features, high_target, cv=5, scoring='neg_mean_absolute_error')
    print(f"Cross-validation MAE for High Target: {-scores_high.mean():.4f}")

    model_low = RandomForestRegressor(n_estimators=200, random_state=42)
    scores_low = cross_val_score(model_low, features, low_target, cv=5, scoring='neg_mean_absolute_error')
    print(f"Cross-validation MAE for Low Target: {-scores_low.mean():.4f}")

    # Latih model
    model_high.fit(features, high_target)
    model_low.fit(features, low_target)

    # Prediksi harga tertinggi dan terendah
    predicted_high = model_high.predict(features.tail(1))[0]
    predicted_low = model_low.predict(features.tail(1))[0]
    return predicted_high, predicted_low

# ================== TAMPILKAN HASIL ==================
if __name__ == "__main__":
    # Dapatkan harga terkini dari CoinGecko
    coin_gecko_usd, coin_gecko_idr = get_live_price_coingecko()

    # Tampilkan harga real-time
    if coin_gecko_usd is not None and coin_gecko_idr is not None:
        print(f"Harga XRP (USD - CoinGecko): ${coin_gecko_usd:.4f}")
        print(f"Harga XRP (IDR - CoinGecko): Rp{coin_gecko_idr:,.0f}".replace(",", "."))
    else:
        print("❌ Gagal mendapatkan harga XRP dari CoinGecko.")

    # Dapatkan data historis dari Yahoo Finance dan Indodax
    yahoo_data = get_historical_data_yahoo(days=30, interval="1d")
    indodax_data = get_historical_data_indodax(days=30)

    if yahoo_data is None:
        print("Failed to fetch historical data from Yahoo Finance.")
        exit()

    if indodax_data is None:
        print("Failed to fetch historical data from Indodax. Using Yahoo Finance data only.")
        combined_data = yahoo_data[['Close', 'Volume']].rename(columns={'Close': 'Close_Yahoo', 'Volume': 'Volume_Yahoo'})
    else:
        combined_data = combine_data(yahoo_data, indodax_data)

    if combined_data.empty:
        print("❌ Combined data is empty. Cannot proceed with prediction.")
        exit()

    # Lakukan prediksi
    predicted_high, predicted_low = predict_tomorrow(combined_data)

    # Format angka Rupiah
    def format_idr(price):
        return f"Rp{price:,.0f}".replace(",", ".")

    # Definisikan warna ANSI
    GREEN = '\033[92m'  # Hijau
    RED = '\033[91m'    # Merah
    RESET = '\033[0m'   # Reset warna

    # Tampilkan hasil
    print("\n" + "="*80)
    print(f"🕒 Update Terkini: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    if coin_gecko_usd is not None:
        print(f"💰 Harga XRP (USD - CoinGecko): ${coin_gecko_usd:,.4f}")
    else:
        print("❌ Gagal mendapatkan harga XRP (USD) dari CoinGecko.")
    if coin_gecko_idr is not None:
        print(f"💰 Harga XRP (IDR - CoinGecko): {format_idr(coin_gecko_idr)}")
    else:
        print("❌ Gagal mendapatkan harga XRP (IDR) dari CoinGecko.")
    if predicted_high is not None and predicted_low is not None:
        print(f"📈 Prediksi Harga Tertinggi Besok: ${predicted_high:.4f} (USD)")
        print(f"📉 Prediksi Harga Terendah Besok: ${predicted_low:.4f} (USD)")
    else:
        print("❌ Gagal melakukan prediksi harga.")
    print("="*80 + "\n")

Harga XRP (USD - CoinGecko): $2.5100
Harga XRP (IDR - CoinGecko): Rp40.993


[*********************100%***********************]  1 of 1 completed


Raw Indodax Data (First 5 Entries):
{'date': '1738731223', 'price': '41243', 'amount': '0.48325776', 'tid': '11028913', 'type': 'buy'}
{'date': '1738731211', 'price': '41237', 'amount': '12.08366273', 'tid': '11028912', 'type': 'buy'}
{'date': '1738731210', 'price': '41238', 'amount': '17.07502788', 'tid': '11028911', 'type': 'buy'}
{'date': '1738731208', 'price': '41213', 'amount': '0.48683667', 'tid': '11028910', 'type': 'sell'}
{'date': '1738731203', 'price': '41240', 'amount': '0.24163433', 'tid': '11028909', 'type': 'buy'}


TypeError: Cannot join tz-naive with tz-aware DatetimeIndex